# 📌 ASOS, AWS(기온,강수,바람,습도,기압) 데이터 전처리

In [340]:
import pandas as pd

### 1.데이터 불러오기

In [341]:
forder_path = "../weather_data/"
file_list = ["asos_data_fin.csv","temperature_data_fin.csv","rainfall_data_fin.csv",
             "wind_data_fin.csv","humidity_data_fin.csv","ap_data_fin.csv"]
df_fin = pd.DataFrame()

for i, file in enumerate(file_list):
    df = pd.read_csv(forder_path+file, index_col=0)

    if i == 0:
        df_fin = pd.concat([df_fin,df], axis=1)
    elif (i != 0) & (i == 2):
        df_fin = pd.concat([df_fin,df.iloc[:,2:]], axis=1)
    elif (i != 0) & (i != 2):
        df_fin = pd.concat([df_fin,df.iloc[:,3:]], axis=1)

### 2.기온 데이터 결측치 채우기

#### 2.1 기온 관련 컬럼 추출

In [342]:
temp_cols = [cols for cols in df_fin.columns if ("temperature" in cols.lower())]
temp_cols.insert(0, 'Datetime')

In [343]:
df_fin[temp_cols].head()

,Datetime,Temperature(°C),Dew point Temperature(°C),108_Observed temperature(°C),108_Average temperature per minute(°C),108_Highest temperature per minute(°C),108_Lowest temperature per minute(°C),112_Observed temperature(°C),112_Average temperature per minute(°C),112_Highest temperature per minute(°C),112_Lowest temperature per minute(°C),119_Observed temperature(°C),119_Average temperature per minute(°C),119_Highest temperature per minute(°C),119_Lowest temperature per minute(°C)
0,2018-01-01 00:00:00,-3.2,-14.9,-3.2,-2.9,-2.6,-3.2,-1.4,-1.3,-1.1,-1.5,-1.8,-1.7,-1.5,-2.1
1,2018-01-01 01:00:00,-3.3,-14.7,-3.3,-3.3,-3.2,-3.5,-1.6,-1.4,-1.4,-1.6,-2.6,-2.0,-1.6,-2.6
2,2018-01-01 02:00:00,-3.7,-14.7,-3.7,-3.4,-3.2,-3.7,-1.9,-1.6,-1.6,-1.9,-3.0,-2.7,-2.6,-3.0
3,2018-01-01 03:00:00,-4.0,-14.4,-4.0,-3.8,-3.8,-4.0,-1.8,-2.0,-1.8,-2.2,-4.2,-3.5,-3.0,-4.2
4,2018-01-01 04:00:00,-4.2,-12.3,-4.2,-4.0,-3.9,-4.3,-2.0,-1.9,-1.8,-2.0,-5.3,-4.7,-4.2,-5.4


#### 2.2 "Temperature" 컬럼에 결측치 -9 또는 -99 행 확인하기

In [344]:
# 결측치의 값으로 -9, -99의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[(df_fin["Temperature(°C)"] == -9) | (df_fin["Temperature(°C)"] == -99)][temp_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 38개 입니다.


In [345]:
for index in missing_index:
    # "Temperature(°C)" 컬럼에 결측값이 있을 경우, 지점번호(108)인 서울 값과 (112)인 인천 값과 (119)인 수원 값의 평균을 사용
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    if (df_fin.at[index, "108_Observed temperature(°C)"] not in [-9, -99]) & (df_fin.at[index, "112_Observed temperature(°C)"] not in [-9, -99]) & (df_fin.at[index, "119_Observed temperature(°C)"] not in [-9, -99]):
        avg_temp = (df_fin.at[index, "108_Observed temperature(°C)"] + df_fin.at[index, "112_Observed temperature(°C)"] + df_fin.at[index, "119_Observed temperature(°C)"]) / 3
        df_fin.at[index, "Temperature(°C)"] = avg_temp
    elif (df_fin.at[index, "108_Observed temperature(°C)"] in [-9, -99]):
        df_fin.at[index, "Temperature(°C)"] = df_fin.at[index, "108_Average temperature per minute(°C)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["Temperature(°C)"] == -9) | (df_fin["Temperature(°C)"] == -99)][temp_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

현재 결측값을 가지는 행의 개수는 0개 입니다


#### 2.3 "Average temperature per minute(°C)", "Highest temperature per minute(°C)", "Lowest temperature per minute(°C)"

##### 2.3.1 "Average temperature per minute(°C)"

In [346]:
# 결측치의 값으로 -9, -99의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[(df_fin["108_Average temperature per minute(°C)"] == -9) | (df_fin["108_Average temperature per minute(°C)"] == -99)][temp_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 42개 입니다.


In [347]:
for index in missing_index:
    # "108_Average temperature per minute(°C)" 컬럼에 결측값이 있을 경우, 지점번호(112)인 인천 값으로 대체
    if (df_fin.at[index, "112_Average temperature per minute(°C)"] not in [-9, -99]):
        df_fin.at[index, "108_Average temperature per minute(°C)"] = df_fin.at[index, "112_Average temperature per minute(°C)"]
    elif (df_fin.at[index, "112_Average temperature per minute(°C)"] in [-9, -99]) & (df_fin.at[index, "119_Average temperature per minute(°C)"] not in [-9, -99]):
        df_fin.at[index, "108_Average temperature per minute(°C)"] = df_fin.at[index, "119_Average temperature per minute(°C)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["108_Average temperature per minute(°C)"] == -9) | (df_fin["108_Average temperature per minute(°C)"] == -99)][temp_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

# "Average atmospheric pressure(hPa)" 컬럼 생성
if len(missing_index) == 0:
    df_fin["Average temperature per minute(°C)"] = df_fin["108_Average temperature per minute(°C)"]
    print("새로운 컬럼을 생성하였습니다.")
else:
    print("추가 전처리가 필요합니다.")

현재 결측값을 가지는 행의 개수는 0개 입니다
새로운 컬럼을 생성하였습니다.


##### 2.3.2 "Highest temperature per minute(°C)"

In [348]:
# 결측치의 값으로 -9, -99의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[(df_fin["108_Highest temperature per minute(°C)"] == -9) | (df_fin["108_Highest temperature per minute(°C)"] == -99)][temp_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 27개 입니다.


In [349]:
for index in missing_index:
    # "108_Average temperature per minute(°C)" 컬럼에 결측값이 있을 경우, 지점번호(112)인 인천 값으로 대체
    if (df_fin.at[index, "112_Highest temperature per minute(°C)"] not in [-9, -99]):
        df_fin.at[index, "108_Highest temperature per minute(°C)"] = df_fin.at[index, "112_Highest temperature per minute(°C)"]
    elif (df_fin.at[index, "112_Highest temperature per minute(°C)"] in [-9, -99]) & (df_fin.at[index, "119_Highest temperature per minute(°C)"] not in [-9, -99]):
        df_fin.at[index, "108_Highest temperature per minute(°C)"] = df_fin.at[index, "119_Highest temperature per minute(°C)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["108_Highest temperature per minute(°C)"] == -9) | (df_fin["108_Highest temperature per minute(°C)"] == -99)][temp_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

# "Average atmospheric pressure(hPa)" 컬럼 생성
if len(missing_index) == 0:
    df_fin["Highest temperature per minute(°C)"] = df_fin["108_Highest temperature per minute(°C)"]
    print("새로운 컬럼을 생성하였습니다.")
else:
    print("추가 전처리가 필요합니다.")

현재 결측값을 가지는 행의 개수는 0개 입니다
새로운 컬럼을 생성하였습니다.


##### 2.3.3. "Lowest temperature per minute(°C)"

In [350]:
# 결측치의 값으로 -9, -99의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[(df_fin["108_Lowest temperature per minute(°C)"] == -9) | (df_fin["108_Lowest temperature per minute(°C)"] == -99)][temp_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 32개 입니다.


In [351]:
for index in missing_index:
    # "108_Average temperature per minute(°C)" 컬럼에 결측값이 있을 경우, 지점번호(112)인 인천 값으로 대체
    if (df_fin.at[index, "112_Lowest temperature per minute(°C)"] not in [-9, -99]):
        df_fin.at[index, "108_Lowest temperature per minute(°C)"] = df_fin.at[index, "112_Lowest temperature per minute(°C)"]
    elif (df_fin.at[index, "112_Lowest temperature per minute(°C)"] in [-9, -99]) & (df_fin.at[index, "119_Lowest temperature per minute(°C)"] not in [-9, -99]):
        df_fin.at[index, "108_Lowest temperature per minute(°C)"] = df_fin.at[index, "119_Lowest temperature per minute(°C)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["108_Lowest temperature per minute(°C)"] == -9) | (df_fin["108_Lowest temperature per minute(°C)"] == -99)][temp_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

# "Average atmospheric pressure(hPa)" 컬럼 생성
if len(missing_index) == 0:
    df_fin["Lowest temperature per minute(°C)"] = df_fin["108_Lowest temperature per minute(°C)"]
    print("새로운 컬럼을 생성하였습니다.")
else:
    print("추가 전처리가 필요합니다.")

현재 결측값을 가지는 행의 개수는 0개 입니다
새로운 컬럼을 생성하였습니다.


#### 2.4 기온 관련 필요없는 컬럼 삭제

In [352]:
df_fin.drop(columns=temp_cols[3:], axis=1, inplace=True)

### 3.강수 데이터 결측치 채우기

#### 3.1 강수 관련 컬럼 추출

In [353]:
rf_cols = [cols for cols in df_fin.columns if ("rainfall" in cols.lower())]
rf_cols.insert(0, 'Datetime')

In [354]:
print(df_fin[df_fin["108_Daily Rainfall(mm)"] < 0][rf_cols].shape)
print(df_fin[df_fin["108_Hourly Rainfall(mm)"] < 0][rf_cols].shape)

(914, 13)
(930, 13)


In [355]:
df_fin[df_fin["108_Daily Rainfall(mm)"] != "108_Hourly Rainfall(mm)"][rf_cols].tail(23)

,Datetime,Rainfall(mm),Daily Rainfall(mm),Daily Rainfall2(mm),108_Daily Rainfall(mm),108_Hourly Rainfall(mm),108_Accumulated Rainfall(mm) for max 60min,112_Daily Rainfall(mm),112_Hourly Rainfall(mm),112_Accumulated Rainfall(mm) for max 60min,119_Daily Rainfall(mm),119_Hourly Rainfall(mm),119_Accumulated Rainfall(mm) for max 60min
52561,2023-12-31 01:00:00,-9.0,0.1,0.1,0.1,0.1,0.2,0.0,0.0,-99.0,0.5,0.5,0.5
52562,2023-12-31 02:00:00,-9.0,0.7,0.7,0.7,0.6,0.6,0.0,0.0,-99.0,1.0,0.5,0.7
52563,2023-12-31 03:00:00,0.8,0.8,0.8,0.8,0.1,0.5,0.0,0.0,-99.0,1.2,0.2,0.5
52564,2023-12-31 04:00:00,-9.0,0.8,0.8,0.8,0.0,0.1,0.0,0.0,-99.0,1.2,0.0,0.2
52565,2023-12-31 05:00:00,-9.0,0.8,0.8,0.8,0.0,0.0,0.0,0.0,-99.0,1.2,0.0,0.0
52566,2023-12-31 06:00:00,0.0,0.8,0.8,0.8,0.0,0.0,0.8,0.8,0.8,1.2,0.0,0.0
52567,2023-12-31 07:00:00,-9.0,1.5,1.5,1.5,0.7,0.7,3.1,2.3,3.0,1.3,0.1,0.1
52568,2023-12-31 08:00:00,-9.0,4.4,4.4,4.4,2.9,3.4,3.1,0.0,2.3,1.4,0.1,0.2
52569,2023-12-31 09:00:00,3.9,4.7,4.7,4.7,0.3,2.9,3.1,0.0,0.0,1.4,0.0,0.1
52570,2023-12-31 10:00:00,-9.0,4.7,4.7,4.7,0.0,0.3,3.1,0.0,0.0,1.4,0.0,0.0


#### 3.2 "Rainfall(mm)" 컬럼에 결측치 -9 또는 -99 행 확인하기

In [356]:
# 결측치의 값으로 -9, -99의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[(df_fin["Rainfall(mm)"] == -9) | (df_fin["Rainfall(mm)"] == -99)][rf_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 46822개 입니다.


In [357]:
for index in missing_index:
    # "Rainfall(mm)" 컬럼에 결측치를 지점번호(108)인 서울 데이터로 변경 
    if df_fin.at[index, "108_Hourly Rainfall(mm)"] not in [-9, -99]:
        df_fin.at[index, "Rainfall(mm)"] = df_fin.at[index, "108_Hourly Rainfall(mm)"]
    # 결측치를 지점번호(112)인 인천과 지점번호(119)인 수원의 평균값으로 변경
    elif (df_fin.at[index, "112_Hourly Rainfall(mm)"] not in [-9, -99]) & (df_fin.at[index, "119_Hourly Rainfall(mm)"] not in [-9, -99]):
        avg_rainfall = (df_fin.at[index, "112_Hourly Rainfall(mm)"] + df_fin.at[index, "119_Hourly Rainfall(mm)"]) / 2
        df_fin.at[index, "Rainfall(mm)"] = avg_rainfall
    # 인천이 결측값을 가지는 경우, 수원 값으로 변경
    elif (df_fin.at[index, "112_Hourly Rainfall(mm)"] in [-9, -99]) & (df_fin.at[index, "119_Hourly Rainfall(mm)"] not in [-9, -99]):
        df_fin.at[index, "Rainfall(mm)"] = df_fin.at[index, "119_Hourly Rainfall(mm)"]
    # 수원이 결측값을 가지는 경우, 인천 값으로 변경
    elif (df_fin.at[index, "112_Hourly Rainfall(mm)"] not in [-9, -99]) & (df_fin.at[index, "119_Hourly Rainfall(mm)"] in [-9, -99]):
        df_fin.at[index, "Rainfall(mm)"] = df_fin.at[index, "112_Hourly Rainfall(mm)"]
    # 위 조건에 만족하지 않는 결측값의 행은 "108_Daily Rainfall(mm)" 컬럼을 활용하여 대체
    else:
        df_fin.at[index, "Rainfall(mm)"] = df_fin.at[index, "108_Daily Rainfall(mm)"] - df_fin.at[(index-1), "108_Daily Rainfall(mm)"]


# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["Rainfall(mm)"] < 0)][rf_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

현재 결측값을 가지는 행의 개수는 5개 입니다


In [358]:
# 마지막 결측값을 가진 행 확인
print(missing_index)
df_fin[rf_cols].loc[42246:42248]

Int64Index([38450, 38575, 38718, 42028, 42247], dtype='int64')


,Datetime,Rainfall(mm),Daily Rainfall(mm),Daily Rainfall2(mm),108_Daily Rainfall(mm),108_Hourly Rainfall(mm),108_Accumulated Rainfall(mm) for max 60min,112_Daily Rainfall(mm),112_Hourly Rainfall(mm),112_Accumulated Rainfall(mm) for max 60min,119_Daily Rainfall(mm),119_Hourly Rainfall(mm),119_Accumulated Rainfall(mm) for max 60min
42246,2022-10-27 06:00:00,0.0,-9.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42247,2022-10-27 07:00:00,-99.0,-9.0,-9.0,-99.0,-99.0,0.0,-99.0,-99.0,0.0,-99.0,-99.0,0.0
42248,2022-10-27 08:00:00,0.0,-9.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [359]:
# "Rainfall(mm)" 컬럼에서 결측값을 가진 행 사이 값의 평균값으로 대체
for index in missing_index:
    df_fin.at[index, "Rainfall(mm)"] = (df_fin.at[(index+1), "Rainfall(mm)"] + df_fin.at[(index-1), "Rainfall(mm)"]) / 2

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["Rainfall(mm)"] < 0)][rf_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

현재 결측값을 가지는 행의 개수는 0개 입니다


#### 3.3 강수 관련 필요없는 컬럼 삭제

In [360]:
rf_cols

['Datetime',
 'Rainfall(mm)',
 'Daily Rainfall(mm)',
 'Daily Rainfall2(mm)',
 '108_Daily Rainfall(mm)',
 '108_Hourly Rainfall(mm)',
 '108_Accumulated Rainfall(mm) for max 60min',
 '112_Daily Rainfall(mm)',
 '112_Hourly Rainfall(mm)',
 '112_Accumulated Rainfall(mm) for max 60min',
 '119_Daily Rainfall(mm)',
 '119_Hourly Rainfall(mm)',
 '119_Accumulated Rainfall(mm) for max 60min']

In [361]:
df_fin.drop(columns=rf_cols[2:], axis=1, inplace=True)

### 4.바람 데이터 결측치 채우기

#### 4.1 풍향 관련 컬럼 추출

In [362]:
wd_cols = [cols for cols in df_fin.columns if ("wind direction" in cols.lower())]
wd_cols.insert(0, 'Datetime')

In [363]:
df_fin[wd_cols].head()

,Datetime,Wind Direction,108_Observed wind direction(deg),108_Maximum wind direction per minute(deg),112_Observed wind direction(deg),112_Maximum wind direction per minute(deg),119_Observed wind direction(deg),119_Maximum wind direction per minute(deg)
0,2018-01-01 00:00:00,11,108.0,289.7,325.3,348.7,294.1,310.4
1,2018-01-01 01:00:00,36,350.4,333.9,348.5,348.5,274.7,298.7
2,2018-01-01 02:00:00,27,262.3,328.6,20.9,7.1,306.3,267.7
3,2018-01-01 03:00:00,29,293.0,283.8,351.2,352.9,265.5,266.4
4,2018-01-01 04:00:00,29,292.0,294.9,350.8,347.2,0.0,169.2


#### 4.2 "Wind Direction" 컬럼 값을 세자리 정수로 변경

In [364]:
df_fin["Wind Direction"] = df_fin["Wind Direction"] * 10

#### 4.3 "Wind Direction" 컬럼에 결측치가 0 이하인 행 확인

In [365]:
# 결측치의 값으로 -9, -99의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[df_fin["Wind Direction"] < 0][wd_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 48개 입니다.


In [366]:
for index in missing_index:
    if (df_fin.at[index, "108_Observed wind direction(deg)"] >= 0):
        df_fin.at[index, "Wind Direction"] = df_fin.at[index, "108_Observed wind direction(deg)"]
    # 서울 값이 결측값일 경우, 지점번호(112)인 인천 값을 사용
    elif (df_fin.at[index, "108_Observed wind direction(deg)"] < 0) & (df_fin.at[index, "112_Observed wind direction(deg)"] >= 0):
        df_fin.at[index, "Wind Direction"] = df_fin.at[index, "112_Observed wind direction(deg)"]
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    elif (df_fin.at[index, "108_Observed wind direction(deg)"] < 0 ) & (df_fin.at[index, "112_Observed wind direction(deg)"] < 0) & (df_fin.at[index, "119_Observed wind direction(deg)"] >= 0):
        df_fin.at[index, "Wind Direction"] = df_fin.at[index, "119_Observed wind direction(deg)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["Wind Direction"] < 0)][wd_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

현재 결측값을 가지는 행의 개수는 0개 입니다


#### 4.4 "Maximum wind direction per minute(deg)" 컬럼 생성을 위해 결측치 0 이하 확인

In [367]:
# 결측치의 값으로 -9, -99의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[df_fin["108_Maximum wind direction per minute(deg)"] < 0][wd_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 95개 입니다.


In [368]:
for index in missing_index:
    # 서울 값이 결측값일 경우, 지점번호(112)인 인천 값을 사용
    if (df_fin.at[index, "112_Maximum wind direction per minute(deg)"] >= 0):
        df_fin.at[index, "108_Maximum wind direction per minute(deg)"] = df_fin.at[index, "112_Maximum wind direction per minute(deg)"]
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    elif (df_fin.at[index, "112_Observed wind direction(deg)"] < 0) & (df_fin.at[index, "119_Observed wind direction(deg)"] >= 0):
        df_fin.at[index, "108_Maximum wind direction per minute(deg)"] = df_fin.at[index, "119_Maximum wind direction per minute(deg)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["Wind Direction"] < 0)][wd_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

# "Maximum wind direction per minute(deg)" 컬럼 생성
if len(missing_index) == 0:
    df_fin["Maximum wind direction per minute(deg)"] = df_fin["108_Maximum wind direction per minute(deg)"]
else:
    print("추가 전처리가 필요합니다.")

현재 결측값을 가지는 행의 개수는 0개 입니다


#### 4.5 풍향 관련 필요없는 컬럼 삭제

In [369]:
df_fin.drop(columns=wd_cols[2:], axis=1, inplace=True)

#### 4.6 풍속 관련 컬럼 추출

In [370]:
ws_cols = [cols for cols in df_fin.columns if ("wind speed" in cols.lower())]
ws_cols.insert(0, 'Datetime')

#### 4.7 "Wind Speed" 컬럼에 결측치 0 이하 인 행 확인 

In [371]:
# 결측치의 값으로 0 이하의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[df_fin["Wind Speed(m/s)"] < 0][ws_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 48개 입니다.


In [372]:
for index in missing_index:
    # "Wind Direction" 컬럼에 결측값이 있을 경우, 지점번호(108)인 서울 값을 사용
    if (df_fin.at[index, "108_Observed wind speed(m/s)"] >= 0):
        df_fin.at[index, "Wind Speed(m/s)"] = df_fin.at[index, "108_Observed wind speed(m/s)"]
    # 서울 값이 결측값일 경우, 지점번호(112)인 인천 값을 사용
    elif (df_fin.at[index, "108_Observed wind speed(m/s)"] < 0) & (df_fin.at[index, "112_Observed wind speed(m/s)"] >= 0):
        df_fin.at[index, "Wind Speed(m/s)"] = df_fin.at[index, "112_Observed wind speed(m/s)"]
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    elif (df_fin.at[index, "108_Observed wind speed(m/s)"] < 0) & (df_fin.at[index, "112_Observed wind speed(m/s)"] < 0) & (df_fin.at[index, "119_Observed wind speed(m/s)"] >= 0):
        df_fin.at[index, "Wind Speed(m/s)"] = df_fin.at[index, "119_Observed wind speed(m/s)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["Wind Speed(m/s)"] < 0)][ws_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

현재 결측값을 가지는 행의 개수는 0개 입니다


#### 4.8 "Average wind speed per minute(m/s)", "Maximum wind speed per minute(m/s)" 컬럼 생성을 위한 결측치 0이하인 행 확인

##### 4.8.1 "Average wind speed per minute(m/s)

In [373]:
# 결측치의 값으로 -9의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[df_fin["108_Average wind speed per minute(m/s)"] < 0][ws_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 95개 입니다.


In [374]:
for index in missing_index:
    # 서울 값이 결측값일 경우, 지점번호(112)인 인천 값을 사용
    if (df_fin.at[index, "112_Average wind speed per minute(m/s)"] >= 0):
        df_fin.at[index, "108_Average wind speed per minute(m/s)"] = df_fin.at[index, "112_Average wind speed per minute(m/s)"]
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    elif (df_fin.at[index, "112_Average wind speed per minute(m/s)"] < 0) & (df_fin.at[index, "119_Average wind speed per minute(m/s)"] >= 0):
        df_fin.at[index, "108_Average wind speed per minute(m/s)"] = df_fin.at[index, "119_Average wind speed per minute(m/s)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["108_Average wind speed per minute(m/s)"] < 0)][ws_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

# "Miximum wind speed per minute(m/s)" 컬럼 생성
if len(missing_index) == 0:
    df_fin["Average wind speed per minute(m/s)"] = df_fin["108_Average wind speed per minute(m/s)"]
    print("새로운 컬럼 생성하였습니다.")
else:
    print("추가 전처리가 필요합니다.")

현재 결측값을 가지는 행의 개수는 0개 입니다
새로운 컬럼 생성하였습니다.


##### 4.8.2 "Maximum wind speed per minute(m/s)"

In [375]:
# 결측치의 값으로 -9의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[df_fin["108_Maximum wind speed per minute(m/s)"] < 0][ws_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 95개 입니다.


In [376]:
for index in missing_index:
    # 서울 값이 결측값일 경우, 지점번호(112)인 인천 값을 사용
    if (df_fin.at[index, "112_Maximum wind speed per minute(m/s)"] >= 0):
        df_fin.at[index, "108_Maximum wind speed per minute(m/s)"] = df_fin.at[index, "112_Maximum wind speed per minute(m/s)"]
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    elif (df_fin.at[index, "112_Maximum wind speed per minute(m/s)"] < 0) & (df_fin.at[index, "119_Maximum wind speed per minute(m/s)"] >= 0):
        df_fin.at[index, "108_Maximum wind speed per minute(m/s)"] = df_fin.at[index, "119_Maximum wind speed per minute(m/s)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["108_Maximum wind speed per minute(m/s)"] < 0)][ws_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

# "Miximum wind speed per minute(m/s)" 컬럼 생성
if len(missing_index) == 0:
    df_fin["Maximum wind speed per minute(m/s)"] = df_fin["108_Maximum wind speed per minute(m/s)"]
    print("새로운 컬럼 생성하였습니다.")
else:
    print("추가 전처리가 필요합니다.")

현재 결측값을 가지는 행의 개수는 0개 입니다
새로운 컬럼 생성하였습니다.


#### 4.9 풍속 관련 필요없는 컬럼 삭제

In [377]:
df_fin[ws_cols].columns

Index(['Datetime', 'Wind Speed(m/s)', '108_Observed wind speed(m/s)',
       '108_Average wind speed per minute(m/s)',
       '108_Maximum wind speed per minute(m/s)',
       '112_Observed wind speed(m/s)',
       '112_Average wind speed per minute(m/s)',
       '112_Maximum wind speed per minute(m/s)',
       '119_Observed wind speed(m/s)',
       '119_Average wind speed per minute(m/s)',
       '119_Maximum wind speed per minute(m/s)'],
      dtype='object')

In [378]:
df_fin.drop(columns=ws_cols[2:], axis=1, inplace=True)

### 5.습도 데이터 결측치 채우기

#### 5.1 습도 관련 컬럼 추출

In [379]:
hum_cols = [cols for cols in df_fin.columns if ("humidity" in cols.lower())]
hum_cols.insert(0, 'Datetime')

#### 5.2 "Relative Humidity(%)" 컬럼 값에 결측치 -9인 행 확인

In [380]:
# 결측치의 값으로 -9 의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[df_fin["Relative Humidity(%)"] < 0][hum_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 19개 입니다.


In [381]:
for index in missing_index:
    # "Relative Humidity" 컬럼에 결측값이 있을 경우, 지점번호(108)인 서울 값을 사용
    if df_fin.at[index, "108_Observed humidity(%)"] not in [-9]:
        df_fin.at[index, "Relative Humidity(%)"] = df_fin.at[index, "108_Observed humidity(%)"]
    # 서울 값이 결측값일 경우, 지점번호(112)인 인천 값을 사용
    elif (df_fin.at[index, "108_Observed humidity(%)"] in [-9]) & (df_fin.at[index, "112_Observed humidity(%)"] not in [-9]):
        df_fin.at[index, "Relative Humidity(%)"] = df_fin.at[index, "112_Observed humidity(%)"]
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    elif (df_fin.at[index, "108_Observed humidity(%)"] in [-9]) & (df_fin.at[index, "112_Observed humidity(%)"] in [-9]) & (df_fin.at[index, "119_Observed humidity(%)"] not in [-9]):
        df_fin.at[index, "Relative Humidity(%)"] = df_fin.at[index, "119_Observed humidity(%)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["Relative Humidity(%)"] == -9)][hum_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

현재 결측값을 가지는 행의 개수는 0개 입니다


#### 5.3 "Average humidity per minute(%)", "Maximum humidity per minute(%)", "Minimum humidity per minute(%)

##### 5.3.1 "Average humidity per minute(%)"

In [382]:
# 결측치의 값으로 -9의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[df_fin["108_Average humidity per minute(%)"] < 0][hum_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 10개 입니다.


In [383]:
for index in missing_index:
    # 서울 값이 결측값일 경우, 지점번호(112)인 인천 값을 사용
    if (df_fin.at[index, "112_Average humidity per minute(%)"] not in [-9]):
        df_fin.at[index, "108_Average humidity per minute(%)"] = df_fin.at[index, "112_Average humidity per minute(%)"]
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    elif (df_fin.at[index, "112_Average humidity per minute(%)"] in [-9]) & (df_fin.at[index, "119_Average humidity per minute(%)"] not in [-9]):
        df_fin.at[index, "108_Average humidity per minute(%))"] = df_fin.at[index, "119_Average humidity per minute(%)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["108_Average humidity per minute(%)"] == -9)][hum_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

# "Average humidity per minute(%)" 컬럼 생성
if len(missing_index) == 0:
    df_fin["Average humidity per minute(%)"] = df_fin["108_Average humidity per minute(%)"]
else:
    print("추가 전처리가 필요합니다.")

현재 결측값을 가지는 행의 개수는 0개 입니다


##### 5.3.2 "Maximum humidity per minute(%)"

In [384]:
# 결측치의 값으로 -9의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[df_fin["108_Maximum humidity per minute(%)"] < 0][hum_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 10개 입니다.


In [385]:
for index in missing_index:
    # 서울 값이 결측값일 경우, 지점번호(112)인 인천 값을 사용
    if (df_fin.at[index, "112_Maximum humidity per minute(%)"] not in [-9]):
        df_fin.at[index, "108_Maximum humidity per minute(%)"] = df_fin.at[index, "112_Maximum humidity per minute(%)"]
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    elif (df_fin.at[index, "112_Maximum humidity per minute(%)"] in [-9]) & (df_fin.at[index, "119_Maximum humidity per minute(%)"] not in [-9]):
        df_fin.at[index, "108_Maximum humidity per minute(%))"] = df_fin.at[index, "119_Maximum humidity per minute(%)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["108_Maximum humidity per minute(%)"] == -9)][hum_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

# "Maximum humidity per minute(%)" 컬럼 생성
if len(missing_index) == 0:
    df_fin["Maximum humidity per minute(%)"] = df_fin["108_Maximum humidity per minute(%)"]
else:
    print("추가 전처리가 필요합니다.")

현재 결측값을 가지는 행의 개수는 0개 입니다


##### 5.3.3 "Minimum humidity per minute(%)"

In [386]:
# 결측치의 값으로 -9의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[df_fin["108_Minimum humidity per minute(%)"] < 0][hum_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 11개 입니다.


In [387]:
for index in missing_index:
    # 서울 값이 결측값일 경우, 지점번호(112)인 인천 값을 사용
    if (df_fin.at[index, "112_Minimum humidity per minute(%)"] not in [-9]):
        df_fin.at[index, "108_Minimum humidity per minute(%)"] = df_fin.at[index, "112_Minimum humidity per minute(%)"]
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    elif (df_fin.at[index, "112_Minimum humidity per minute(%)"] in [-9]) & (df_fin.at[index, "119_Minimum humidity per minute(%)"] not in [-9]):
        df_fin.at[index, "108_Minimum humidity per minute(%))"] = df_fin.at[index, "119_Minimum humidity per minute(%)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["108_Minimum humidity per minute(%)"] == -9)][hum_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

# "Minimum humidity per minute(%)" 컬럼 생성
if len(missing_index) == 0:
    df_fin["Minimum humidity per minute(%)"] = df_fin["108_Maximum humidity per minute(%)"]
else:
    print("추가 전처리가 필요합니다.")

현재 결측값을 가지는 행의 개수는 0개 입니다


#### 5.4 습도 관련 필요없는 컬럼 삭제

In [388]:
df_fin[hum_cols].columns

Index(['Datetime', 'Relative Humidity(%)', '108_Observed humidity(%)',
       '108_Average humidity per minute(%)',
       '108_Maximum humidity per minute(%)',
       '108_Minimum humidity per minute(%)', '112_Observed humidity(%)',
       '112_Average humidity per minute(%)',
       '112_Maximum humidity per minute(%)',
       '112_Minimum humidity per minute(%)', '119_Observed humidity(%)',
       '119_Average humidity per minute(%)',
       '119_Maximum humidity per minute(%)',
       '119_Minimum humidity per minute(%)'],
      dtype='object')

In [389]:
df_fin.drop(columns=hum_cols[2:], axis=1, inplace=True)

### 6.기압 데이터 결측치 채우기

#### 6.1 기압 관련 컬럼 추출

In [390]:
ap_cols = [cols for cols in df_fin.columns if ("atmospheric pressure" in cols.lower())]
ap_cols.insert(0, 'Datetime')

In [391]:
ap_cols

['Datetime',
 'Atmospheric Pressure(hPa)',
 '108_Observed atmospheric pressure(hPa)',
 '108_Average atmospheric pressure(hPa)',
 '108_Highest atmospheric pressure(hPa)',
 '108_Minimum atmospheric pressure(hPa)',
 '112_Observed atmospheric pressure(hPa)',
 '112_Average atmospheric pressure(hPa)',
 '112_Highest atmospheric pressure(hPa)',
 '112_Minimum atmospheric pressure(hPa)',
 '119_Observed atmospheric pressure(hPa)',
 '119_Average atmospheric pressure(hPa)',
 '119_Highest atmospheric pressure(hPa)',
 '119_Minimum atmospheric pressure(hPa)']

#### 6.2 "Atmospheric Pressure(hPa)" 컬럼 값에 결측치 -9, -99인 행 확인

In [392]:
# 결측치의 값으로 -9 의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[df_fin["Atmospheric Pressure(hPa)"] < 0][ap_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 29개 입니다.


In [393]:
for index in missing_index:
    # "Atmospheric Pressure(hPa)" 컬럼에 결측값이 있을 경우, 지점번호(108)인 서울 값을 사용
    if df_fin.at[index, "108_Observed atmospheric pressure(hPa)"] not in [-9, -99]:
        df_fin.at[index, "Atmospheric Pressure(hPa)"] = df_fin.at[index, "108_Observed atmospheric pressure(hPa)"]
    # 서울 값이 결측값일 경우, 지점번호(112)인 인천 값을 사용
    elif (df_fin.at[index, "108_Observed atmospheric pressure(hPa)"] in [-9, -99]) & (df_fin.at[index, "112_Observed atmospheric pressure(hPa)"] not in [-9, -99]):
        df_fin.at[index, "Atmospheric Pressure(hPa)"] = df_fin.at[index, "112_Observed atmospheric pressure(hPa)"]
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    elif (df_fin.at[index, "108_Observed atmospheric pressure(hPa)"] in [-9, -99]) & (df_fin.at[index, "112_Observed atmospheric pressure(hPa)"] in [-9, -99]) & (df_fin.at[index, "119_Observed atmospheric pressure(hPa)"] not in [-9, -99]):
        df_fin.at[index, "Atmospheric Pressure(hPa)"] = df_fin.at[index, "119_Observed atmospheric pressure(hPa)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["Atmospheric Pressure(hPa)"] < 0)][ap_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

현재 결측값을 가지는 행의 개수는 0개 입니다


#### 6.3 "Average atmospheric pressure(hPa)", "Highest atmospheric pressure(hPa)", "Minimum atmospheric pressure(hPa)"

##### 6.3.1 "Average atmospheric pressure(hPa)"

In [394]:
# 결측치의 값으로 -9 의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[df_fin["108_Average atmospheric pressure(hPa)"] < 0][ap_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 19개 입니다.


In [395]:
for index in missing_index:
    # 서울 값이 결측값일 경우, 지점번호(112)인 인천 값을 사용
    if (df_fin.at[index, "112_Average atmospheric pressure(hPa)"] not in [-9, -99]):
        df_fin.at[index, "108_Average atmospheric pressure(hPa)"] = df_fin.at[index, "112_Average atmospheric pressure(hPa)"]
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    elif (df_fin.at[index, "112_Average atmospheric pressure(hPa)"] in [-9, -99]) & (df_fin.at[index, "119_Average atmospheric pressure(hPa)"] not in [-9, -99]):
        df_fin.at[index, "108_Average atmospheric pressure(hPa)"] = df_fin.at[index, "119_Average atmospheric pressure(hPa)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["108_Average atmospheric pressure(hPa)"] < 0)][ap_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

# "Average atmospheric pressure(hPa)" 컬럼 생성
if len(missing_index) == 0:
    df_fin["Average atmospheric pressure(hPa)"] = df_fin["108_Average atmospheric pressure(hPa)"]
    print("새로운 컬럼을 생성하였습니다.")
else:
    print("추가 전처리가 필요합니다.")

현재 결측값을 가지는 행의 개수는 0개 입니다
새로운 컬럼을 생성하였습니다.


##### 6.3.2 "Highest atmospheric pressure(hPa)"

In [396]:
# 결측치의 값으로 -9 의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[df_fin["108_Highest atmospheric pressure(hPa)"] < 0][ap_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 19개 입니다.


In [397]:
for index in missing_index:
    # 서울 값이 결측값일 경우, 지점번호(112)인 인천 값을 사용
    if (df_fin.at[index, "112_Highest atmospheric pressure(hPa)"] not in [-9, -99]):
        df_fin.at[index, "108_Highest atmospheric pressure(hPa)"] = df_fin.at[index, "112_Highest atmospheric pressure(hPa)"]
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    elif (df_fin.at[index, "112_Highest atmospheric pressure(hPa)"] in [-9, -99]) & (df_fin.at[index, "119_Highest atmospheric pressure(hPa)"] not in [-9, -99]):
        df_fin.at[index, "108_Highest atmospheric pressure(hPa)"] = df_fin.at[index, "119_Highest atmospheric pressure(hPa)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["108_Highest atmospheric pressure(hPa)"] < 0)][ap_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

# "Average atmospheric pressure(hPa)" 컬럼 생성
if len(missing_index) == 0:
    df_fin["Highest atmospheric pressure(hPa)"] = df_fin["108_Highest atmospheric pressure(hPa)"]
    print("새로운 컬럼을 생성하였습니다.")
else:
    print("추가 전처리가 필요합니다.")

현재 결측값을 가지는 행의 개수는 0개 입니다
새로운 컬럼을 생성하였습니다.


##### 6.3.3 "Minimum atmospheric pressure(hPa)"

In [398]:
# 결측치의 값으로 -9 의 값을 가지는 행 번호를 가지는 변수 생성
missing_index = df_fin[df_fin["108_Minimum atmospheric pressure(hPa)"] < 0][ap_cols].index
# 인덱스의 개수를 확인하기 위한 출력
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다.")

현재 결측값을 가지는 행의 개수는 19개 입니다.


In [399]:
for index in missing_index:
    # 서울 값이 결측값일 경우, 지점번호(112)인 인천 값을 사용
    if (df_fin.at[index, "112_Minimum atmospheric pressure(hPa)"] not in [-9, -99]):
        df_fin.at[index, "108_Minimum atmospheric pressure(hPa)"] = df_fin.at[index, "112_Minimum atmospheric pressure(hPa)"]
    # 인천 값이 결측값일 경우, 지점번호(119) 수원 값을 사용
    elif (df_fin.at[index, "112_Minimum atmospheric pressure(hPa)"] in [-9, -99]) & (df_fin.at[index, "119_Minimum atmospheric pressure(hPa)"] not in [-9, -99]):
        df_fin.at[index, "108_Minimum atmospheric pressure(hPa)"] = df_fin.at[index, "119_Minimum atmospheric pressure(hPa)"]

# 인덱스의 개수를 확인하기 위한 출력
missing_index = df_fin[(df_fin["108_Minimum atmospheric pressure(hPa)"] < 0)][ap_cols].index
print(f"현재 결측값을 가지는 행의 개수는 {len(missing_index)}개 입니다")

# "Average atmospheric pressure(hPa)" 컬럼 생성
if len(missing_index) == 0:
    df_fin["Minimum atmospheric pressure(hPa)"] = df_fin["108_Minimum atmospheric pressure(hPa)"]
    print("새로운 컬럼을 생성하였습니다.")
else:
    print("추가 전처리가 필요합니다.")

현재 결측값을 가지는 행의 개수는 0개 입니다
새로운 컬럼을 생성하였습니다.


#### 6.4 기압 관련 필요없는 컬럼 삭제

In [400]:
df_fin[ap_cols].columns

Index(['Datetime', 'Atmospheric Pressure(hPa)',
       '108_Observed atmospheric pressure(hPa)',
       '108_Average atmospheric pressure(hPa)',
       '108_Highest atmospheric pressure(hPa)',
       '108_Minimum atmospheric pressure(hPa)',
       '112_Observed atmospheric pressure(hPa)',
       '112_Average atmospheric pressure(hPa)',
       '112_Highest atmospheric pressure(hPa)',
       '112_Minimum atmospheric pressure(hPa)',
       '119_Observed atmospheric pressure(hPa)',
       '119_Average atmospheric pressure(hPa)',
       '119_Highest atmospheric pressure(hPa)',
       '119_Minimum atmospheric pressure(hPa)'],
      dtype='object')

In [401]:
df_fin.drop(columns=ap_cols[2:], axis=1, inplace=True)

### 7.그 외 데이터 결측치 채우기
- 'Dew point Temperature(°C)'
- '3hr Snowfall(mm)', 'Daily Snowfall(mm)', 'Daily Sonwfall2(mm)'

In [402]:
df_fin.columns

Index(['Datetime', 'Date', 'Hour', 'Wind Direction', 'Wind Speed(m/s)',
       'Atmospheric Pressure(hPa)', 'Temperature(°C)',
       'Dew point Temperature(°C)', 'Relative Humidity(%)', 'Rainfall(mm)',
       '3hr Snowfall(mm)', 'Daily Snowfall(mm)', 'Daily Sonwfall2(mm)',
       'Average temperature per minute(°C)',
       'Highest temperature per minute(°C)',
       'Lowest temperature per minute(°C)',
       'Maximum wind direction per minute(deg)',
       'Average wind speed per minute(m/s)',
       'Maximum wind speed per minute(m/s)', 'Average humidity per minute(%)',
       'Maximum humidity per minute(%)', 'Minimum humidity per minute(%)',
       'Average atmospheric pressure(hPa)',
       'Highest atmospheric pressure(hPa)',
       'Minimum atmospheric pressure(hPa)'],
      dtype='object')

#### 7.1 컬럼 추출

In [403]:
print(df_fin[df_fin["Dew point Temperature(°C)"] == -9].shape)
print(df_fin[df_fin["3hr Snowfall(mm)"] == -9].shape)
print(df_fin[df_fin["Daily Snowfall(mm)"] == -9].shape)
print(df_fin[df_fin["Daily Sonwfall2(mm)"] == -9].shape)

(116, 25)
(52442, 25)
(51987, 25)
(50902, 25)


#### 7.2 필요없는 컬럼 삭제

In [404]:
df_fin.drop(["Dew point Temperature(°C)", "3hr Snowfall(mm)", 
             "Daily Snowfall(mm)", "Daily Sonwfall2(mm)"], axis=1, inplace=True)

In [405]:
print(df_fin.columns)
print(df_fin.shape)

Index(['Datetime', 'Date', 'Hour', 'Wind Direction', 'Wind Speed(m/s)',
       'Atmospheric Pressure(hPa)', 'Temperature(°C)', 'Relative Humidity(%)',
       'Rainfall(mm)', 'Average temperature per minute(°C)',
       'Highest temperature per minute(°C)',
       'Lowest temperature per minute(°C)',
       'Maximum wind direction per minute(deg)',
       'Average wind speed per minute(m/s)',
       'Maximum wind speed per minute(m/s)', 'Average humidity per minute(%)',
       'Maximum humidity per minute(%)', 'Minimum humidity per minute(%)',
       'Average atmospheric pressure(hPa)',
       'Highest atmospheric pressure(hPa)',
       'Minimum atmospheric pressure(hPa)'],
      dtype='object')
(52584, 21)


### 8.대여건수 데이터와 요일 데이터 추가하기 

#### 8.1 대여건수 데이터 불러오기

In [406]:
# rental_history_data.csv 파일 불러오기
rental_df = pd.read_csv("../data/rental_history_data.csv", index_col=0)
rental_df

,Date,Hour,Rent Bike Count
0,2018-01-01,0,268
1,2018-01-01,1,304
2,2018-01-01,2,228
3,2018-01-01,3,148
4,2018-01-01,4,85
...,...,...,...
52579,2023-12-31,19,799
52580,2023-12-31,20,844
52581,2023-12-31,21,850
52582,2023-12-31,22,818


In [407]:
# 데이터프레임 합치기
df_fin = pd.concat([df_fin, rental_df["Rent Bike Count"]], axis=1)
df_fin.head()

,Datetime,Date,Hour,Wind Direction,Wind Speed(m/s),Atmospheric Pressure(hPa),Temperature(°C),Relative Humidity(%),Rainfall(mm),Average temperature per minute(°C),...,Maximum wind direction per minute(deg),Average wind speed per minute(m/s),Maximum wind speed per minute(m/s),Average humidity per minute(%),Maximum humidity per minute(%),Minimum humidity per minute(%),Average atmospheric pressure(hPa),Highest atmospheric pressure(hPa),Minimum atmospheric pressure(hPa),Rent Bike Count
0,2018-01-01 00:00:00,2018-01-01,0,110.0,0.5,1015.4,-3.2,40.0,0.0,-2.9,...,289.7,0.5,1.4,39.9,41.6,41.6,1015.2,1015.4,1015.1,268
1,2018-01-01 01:00:00,2018-01-01,1,360.0,0.7,1015.1,-3.3,41.0,0.0,-3.3,...,333.9,0.6,1.7,40.3,41.6,41.6,1015.3,1015.5,1015.1,304
2,2018-01-01 02:00:00,2018-01-01,2,270.0,0.9,1015.2,-3.7,42.0,0.0,-3.4,...,328.6,0.7,1.6,41.4,45.9,45.9,1015.0,1015.2,1014.9,228
3,2018-01-01 03:00:00,2018-01-01,3,290.0,1.0,1015.5,-4.0,44.0,0.0,-3.8,...,283.8,0.8,1.4,44.0,46.6,46.6,1015.4,1015.6,1015.2,148
4,2018-01-01 04:00:00,2018-01-01,4,290.0,1.1,1015.5,-4.2,53.0,0.0,-4.0,...,294.9,0.8,1.2,49.3,52.7,52.7,1015.5,1015.5,1015.5,85


In [408]:
df_fin.columns

Index(['Datetime', 'Date', 'Hour', 'Wind Direction', 'Wind Speed(m/s)',
       'Atmospheric Pressure(hPa)', 'Temperature(°C)', 'Relative Humidity(%)',
       'Rainfall(mm)', 'Average temperature per minute(°C)',
       'Highest temperature per minute(°C)',
       'Lowest temperature per minute(°C)',
       'Maximum wind direction per minute(deg)',
       'Average wind speed per minute(m/s)',
       'Maximum wind speed per minute(m/s)', 'Average humidity per minute(%)',
       'Maximum humidity per minute(%)', 'Minimum humidity per minute(%)',
       'Average atmospheric pressure(hPa)',
       'Highest atmospheric pressure(hPa)',
       'Minimum atmospheric pressure(hPa)', 'Rent Bike Count'],
      dtype='object')

#### 8.2 계절, 요일 데이터 추가하기 - "Seasons", "Day of the week", "Weekend", "Holiday"

##### 8.2.1 "Datetime" 컬럼을 "Year", "Month", "Day" 컬럼으로 나누기

In [409]:
date_split = df_fin["Date"].str.split("-")

df_fin["Year"] = date_split.str.get(0)
df_fin["Month"] = date_split.str.get(1)
df_fin["Day"] = date_split.str.get(2)

In [410]:
df_fin.tail()

,Datetime,Date,Hour,Wind Direction,Wind Speed(m/s),Atmospheric Pressure(hPa),Temperature(°C),Relative Humidity(%),Rainfall(mm),Average temperature per minute(°C),...,Average humidity per minute(%),Maximum humidity per minute(%),Minimum humidity per minute(%),Average atmospheric pressure(hPa),Highest atmospheric pressure(hPa),Minimum atmospheric pressure(hPa),Rent Bike Count,Year,Month,Day
52579,2023-12-31 19:00:00,2023-12-31,19,250.0,0.8,1012.0,2.6,90.0,0.0,2.8,...,89.7,91.3,91.3,1011.6,1012.0,1011.2,799,2023,12,31
52580,2023-12-31 20:00:00,2023-12-31,20,250.0,2.2,1013.0,1.9,94.0,0.0,2.1,...,91.5,94.3,94.3,1012.4,1013.0,1012.0,844,2023,12,31
52581,2023-12-31 21:00:00,2023-12-31,21,250.0,2.4,1013.7,1.7,96.0,0.0,1.7,...,94.2,96.1,96.1,1013.3,1013.7,1013.0,850,2023,12,31
52582,2023-12-31 22:00:00,2023-12-31,22,340.0,0.6,1014.1,1.3,95.0,0.0,1.5,...,95.6,96.8,96.8,1013.8,1014.1,1013.6,818,2023,12,31
52583,2023-12-31 23:00:00,2023-12-31,23,70.0,0.8,1014.9,0.9,96.0,0.0,1.0,...,95.4,96.6,96.6,1014.5,1014.9,1014.1,875,2023,12,31


##### 8.2.2 "Seasons" 컬럼 생성

In [411]:
df_fin['Seasons'] = df_fin['Month'].astype(int)

def map_season(month):
    if 3 <= month < 6:
        return 'Spring'
    elif 6 <= month < 9:
        return 'Summer'
    elif 9 <= month < 12:
        return 'Autumn'
    else:
        return 'Winter'

In [412]:
df_fin['Seasons'] = df_fin['Seasons'].apply(map_season)
df_fin.head()

,Datetime,Date,Hour,Wind Direction,Wind Speed(m/s),Atmospheric Pressure(hPa),Temperature(°C),Relative Humidity(%),Rainfall(mm),Average temperature per minute(°C),...,Maximum humidity per minute(%),Minimum humidity per minute(%),Average atmospheric pressure(hPa),Highest atmospheric pressure(hPa),Minimum atmospheric pressure(hPa),Rent Bike Count,Year,Month,Day,Seasons
0,2018-01-01 00:00:00,2018-01-01,0,110.0,0.5,1015.4,-3.2,40.0,0.0,-2.9,...,41.6,41.6,1015.2,1015.4,1015.1,268,2018,01,01,Winter
1,2018-01-01 01:00:00,2018-01-01,1,360.0,0.7,1015.1,-3.3,41.0,0.0,-3.3,...,41.6,41.6,1015.3,1015.5,1015.1,304,2018,01,01,Winter
2,2018-01-01 02:00:00,2018-01-01,2,270.0,0.9,1015.2,-3.7,42.0,0.0,-3.4,...,45.9,45.9,1015.0,1015.2,1014.9,228,2018,01,01,Winter
3,2018-01-01 03:00:00,2018-01-01,3,290.0,1.0,1015.5,-4.0,44.0,0.0,-3.8,...,46.6,46.6,1015.4,1015.6,1015.2,148,2018,01,01,Winter
4,2018-01-01 04:00:00,2018-01-01,4,290.0,1.1,1015.5,-4.2,53.0,0.0,-4.0,...,52.7,52.7,1015.5,1015.5,1015.5,85,2018,01,01,Winter


##### 8.2.3 "Day of the week" 컬럼 생성

In [413]:
df_fin['Day of Week'] = pd.to_datetime(df_fin['Date']).dt.day_name()
df_fin.head()

,Datetime,Date,Hour,Wind Direction,Wind Speed(m/s),Atmospheric Pressure(hPa),Temperature(°C),Relative Humidity(%),Rainfall(mm),Average temperature per minute(°C),...,Minimum humidity per minute(%),Average atmospheric pressure(hPa),Highest atmospheric pressure(hPa),Minimum atmospheric pressure(hPa),Rent Bike Count,Year,Month,Day,Seasons,Day of Week
0,2018-01-01 00:00:00,2018-01-01,0,110.0,0.5,1015.4,-3.2,40.0,0.0,-2.9,...,41.6,1015.2,1015.4,1015.1,268,2018,01,01,Winter,Monday
1,2018-01-01 01:00:00,2018-01-01,1,360.0,0.7,1015.1,-3.3,41.0,0.0,-3.3,...,41.6,1015.3,1015.5,1015.1,304,2018,01,01,Winter,Monday
2,2018-01-01 02:00:00,2018-01-01,2,270.0,0.9,1015.2,-3.7,42.0,0.0,-3.4,...,45.9,1015.0,1015.2,1014.9,228,2018,01,01,Winter,Monday
3,2018-01-01 03:00:00,2018-01-01,3,290.0,1.0,1015.5,-4.0,44.0,0.0,-3.8,...,46.6,1015.4,1015.6,1015.2,148,2018,01,01,Winter,Monday
4,2018-01-01 04:00:00,2018-01-01,4,290.0,1.1,1015.5,-4.2,53.0,0.0,-4.0,...,52.7,1015.5,1015.5,1015.5,85,2018,01,01,Winter,Monday


##### 8.2.4 "Weekend" 컬럼 생성

In [414]:
def map_weekend(x):
    if x in ['Saturday', 'Sunday']:
        return 'Weekend'
    else:
        return 'Weekdays'

In [415]:
df_fin['Weekend'] = df_fin['Day of Week'].apply(lambda x: map_weekend(x))
df_fin.head()

,Datetime,Date,Hour,Wind Direction,Wind Speed(m/s),Atmospheric Pressure(hPa),Temperature(°C),Relative Humidity(%),Rainfall(mm),Average temperature per minute(°C),...,Average atmospheric pressure(hPa),Highest atmospheric pressure(hPa),Minimum atmospheric pressure(hPa),Rent Bike Count,Year,Month,Day,Seasons,Day of Week,Weekend
0,2018-01-01 00:00:00,2018-01-01,0,110.0,0.5,1015.4,-3.2,40.0,0.0,-2.9,...,1015.2,1015.4,1015.1,268,2018,01,01,Winter,Monday,Weekdays
1,2018-01-01 01:00:00,2018-01-01,1,360.0,0.7,1015.1,-3.3,41.0,0.0,-3.3,...,1015.3,1015.5,1015.1,304,2018,01,01,Winter,Monday,Weekdays
2,2018-01-01 02:00:00,2018-01-01,2,270.0,0.9,1015.2,-3.7,42.0,0.0,-3.4,...,1015.0,1015.2,1014.9,228,2018,01,01,Winter,Monday,Weekdays
3,2018-01-01 03:00:00,2018-01-01,3,290.0,1.0,1015.5,-4.0,44.0,0.0,-3.8,...,1015.4,1015.6,1015.2,148,2018,01,01,Winter,Monday,Weekdays
4,2018-01-01 04:00:00,2018-01-01,4,290.0,1.1,1015.5,-4.2,53.0,0.0,-4.0,...,1015.5,1015.5,1015.5,85,2018,01,01,Winter,Monday,Weekdays


##### 8.2.5 "Holiday" 컬럼 생성

In [416]:
import holidays

hol = holidays.KR()

df_fin['Holiday'] = df_fin['Date'].apply(lambda x: 'Holiday' if x in hol else 'No Holiday')
df_fin.head()

,Datetime,Date,Hour,Wind Direction,Wind Speed(m/s),Atmospheric Pressure(hPa),Temperature(°C),Relative Humidity(%),Rainfall(mm),Average temperature per minute(°C),...,Highest atmospheric pressure(hPa),Minimum atmospheric pressure(hPa),Rent Bike Count,Year,Month,Day,Seasons,Day of Week,Weekend,Holiday
0,2018-01-01 00:00:00,2018-01-01,0,110.0,0.5,1015.4,-3.2,40.0,0.0,-2.9,...,1015.4,1015.1,268,2018,01,01,Winter,Monday,Weekdays,Holiday
1,2018-01-01 01:00:00,2018-01-01,1,360.0,0.7,1015.1,-3.3,41.0,0.0,-3.3,...,1015.5,1015.1,304,2018,01,01,Winter,Monday,Weekdays,Holiday
2,2018-01-01 02:00:00,2018-01-01,2,270.0,0.9,1015.2,-3.7,42.0,0.0,-3.4,...,1015.2,1014.9,228,2018,01,01,Winter,Monday,Weekdays,Holiday
3,2018-01-01 03:00:00,2018-01-01,3,290.0,1.0,1015.5,-4.0,44.0,0.0,-3.8,...,1015.6,1015.2,148,2018,01,01,Winter,Monday,Weekdays,Holiday
4,2018-01-01 04:00:00,2018-01-01,4,290.0,1.1,1015.5,-4.2,53.0,0.0,-4.0,...,1015.5,1015.5,85,2018,01,01,Winter,Monday,Weekdays,Holiday


##### 8.2.5 컬럼 순서 변경

In [417]:
df_fin.columns

Index(['Datetime', 'Date', 'Hour', 'Wind Direction', 'Wind Speed(m/s)',
       'Atmospheric Pressure(hPa)', 'Temperature(°C)', 'Relative Humidity(%)',
       'Rainfall(mm)', 'Average temperature per minute(°C)',
       'Highest temperature per minute(°C)',
       'Lowest temperature per minute(°C)',
       'Maximum wind direction per minute(deg)',
       'Average wind speed per minute(m/s)',
       'Maximum wind speed per minute(m/s)', 'Average humidity per minute(%)',
       'Maximum humidity per minute(%)', 'Minimum humidity per minute(%)',
       'Average atmospheric pressure(hPa)',
       'Highest atmospheric pressure(hPa)',
       'Minimum atmospheric pressure(hPa)', 'Rent Bike Count', 'Year', 'Month',
       'Day', 'Seasons', 'Day of Week', 'Weekend', 'Holiday'],
      dtype='object')

In [418]:
# 컬럼 순서 변경
df_fin = df_fin[['Datetime', 'Date', 'Year', 'Month', 'Day', 'Hour', 'Rent Bike Count',
                 'Temperature(°C)', 'Average temperature per minute(°C)', 
                 'Highest temperature per minute(°C)', 'Lowest temperature per minute(°C)',
                 'Rainfall(mm)', 'Wind Direction', 'Maximum wind direction per minute(deg)', 
                 'Wind Speed(m/s)', 'Average wind speed per minute(m/s)', 'Maximum wind speed per minute(m/s)',
                 'Relative Humidity(%)', 'Average humidity per minute(%)',
                 'Maximum humidity per minute(%)', 'Minimum humidity per minute(%)',
                 'Atmospheric Pressure(hPa)', 'Average atmospheric pressure(hPa)', 
                 'Highest atmospheric pressure(hPa)', 'Minimum atmospheric pressure(hPa)',
                 'Seasons', 'Day of Week', 'Weekend', 'Holiday']]

In [419]:
df_fin.head()

,Datetime,Date,Year,Month,Day,Hour,Rent Bike Count,Temperature(°C),Average temperature per minute(°C),Highest temperature per minute(°C),...,Maximum humidity per minute(%),Minimum humidity per minute(%),Atmospheric Pressure(hPa),Average atmospheric pressure(hPa),Highest atmospheric pressure(hPa),Minimum atmospheric pressure(hPa),Seasons,Day of Week,Weekend,Holiday
0,2018-01-01 00:00:00,2018-01-01,2018,01,01,0,268,-3.2,-2.9,-2.6,...,41.6,41.6,1015.4,1015.2,1015.4,1015.1,Winter,Monday,Weekdays,Holiday
1,2018-01-01 01:00:00,2018-01-01,2018,01,01,1,304,-3.3,-3.3,-3.2,...,41.6,41.6,1015.1,1015.3,1015.5,1015.1,Winter,Monday,Weekdays,Holiday
2,2018-01-01 02:00:00,2018-01-01,2018,01,01,2,228,-3.7,-3.4,-3.2,...,45.9,45.9,1015.2,1015.0,1015.2,1014.9,Winter,Monday,Weekdays,Holiday
3,2018-01-01 03:00:00,2018-01-01,2018,01,01,3,148,-4.0,-3.8,-3.8,...,46.6,46.6,1015.5,1015.4,1015.6,1015.2,Winter,Monday,Weekdays,Holiday
4,2018-01-01 04:00:00,2018-01-01,2018,01,01,4,85,-4.2,-4.0,-3.9,...,52.7,52.7,1015.5,1015.5,1015.5,1015.5,Winter,Monday,Weekdays,Holiday


### 9.데이터프레임을 csv 파일로 저장하기

#### 9.1 전체 데이터프레임을 csv 파일로 저장

In [420]:
df_fin.to_csv("../data/data_fin.csv")

#### 9.2 train, test dataset 분류
- 조건 1 : 대여건수가 0인 데이터만 test_data
- 조건 2 : 대여건수가 200 이하인 데이터만 test_data

In [421]:
print(df_fin[df_fin["Rent Bike Count"] == 0]["Month"].value_counts())
print(df_fin[df_fin["Rent Bike Count"] <= 200]["Month"].value_counts())

08    1117
03     506
04     478
01     466
06     227
09      45
10       7
07       1
Name: Month, dtype: int64
08    1262
01    1103
03     787
04     744
02     535
06     337
05     244
12     180
09     177
07     159
11     107
10      87
Name: Month, dtype: int64


In [422]:
print(len(df_fin[df_fin["Rent Bike Count"] == 0]["Month"]))
print(len(df_fin[df_fin["Rent Bike Count"] <= 200]["Month"]))

2847
5722


##### 9.2.1 대여건수가 0인 조건을 통해 train, test dataset 분류

In [423]:
test_data = df_fin[df_fin["Rent Bike Count"] == 0]
train_data = df_fin[df_fin["Rent Bike Count"] != 0]

test_data.to_csv("../data/test_data/test_data(0).csv", index=False)
train_data.to_csv("../data/train_data/train_data(0).csv", index=False)

##### 9.2.2 대여건수가 200 이하인 조건을 통해 train, test dataset 분류

In [424]:
test_data = df_fin[df_fin["Rent Bike Count"] <=200]
train_data = df_fin[df_fin["Rent Bike Count"] > 200]

test_data.to_csv("../data/test_data/test_data(200).csv", index=False)
train_data.to_csv("../data/train_data/train_data(200).csv", index=False)